<a href="https://colab.research.google.com/github/aj1365/DeepForestPaper/blob/main/2DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
######################################### Connecting to Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/DForest

In [ ]:
####################################### importing libraries for reading matlab files

from scipy import io
import numpy as np

In [ ]:
####################################### importing matlab images as cell files

C = io.loadmat('DataX.mat')['TrainX']

##print( type(C))
#print( C.keys())
#C = C['X_train']
print (type(C))

In [ ]:
#################################### reshaping images
C = np.squeeze(C)
X = np.empty((C.shape[0], C[0].shape[0], C[0].shape[1],C[0].shape[2]))

for i in range(X.shape[0]):
    X[i] = C[i]

features=X
features.shape

In [ ]:
#################################################### Visualize each patch of 7 by 7 
import matplotlib.pyplot as plt
#fig, ax=plt.subplots(figsize=(10,10))
#ax.imshow(X_Train[0,:,:,0])

fig = plt.figure(figsize = (20, 12))

ax = fig.subplots(2, 4)
c=0

for i in range(2):
  for j in range(4):
    ax[i][j].imshow(features[0,:,:,c], cmap='nipy_spectral')
    ax[i][j].axis('off')
    ax[i][j].title.set_text(f"Band - {c}")
    c+=1

plt.tight_layout()
plt.show()

In [ ]:
C2 = io.loadmat('DataY.mat')['TrainY']

print (type(C2))
Labels=C2
Labels.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features,
                                                   Labels,
                                                    stratify=Labels,
                                                    test_size=0.50,
                                                    random_state=0)
y_train=y_train-1
y_test=y_test-1

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train,
                                                   y_train,
                                                    stratify=y_train,
                                                    test_size=0.3,
                                                    random_state=0)

In [ ]:
X_train.shape, X_validation.shape

In [ ]:
pip install plotly

In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, LeakyReLU, MaxPooling2D, Dropout

In [ ]:
###################################### Creating the CNN model

model = Sequential()
model.add(Conv2D(64, kernel_size=(2, 2),activation='linear',padding='same',input_shape=(7,7,28)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(256, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(512, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(20, activation='linear'))
model.add(LeakyReLU(alpha=0.1))           
model.add(Dropout(0.3))
model.add(Dense(8, activation='softmax'))
model.summary()


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', # Loss
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from time import time

In [ ]:
early_stop = EarlyStopping(monitor = 'val_loss',
                            mode = 'min',
                            min_delta = 0,
                            patience = 10,
                            restore_best_weights = True)

checkpoint = ModelCheckpoint(filepath = 'CNN.h11', 
                             monitor = 'val_loss', 
                             mode ='min', 
                             save_best_only = True)

tensorboard = TensorBoard(log_dir='SA_logs/{}'.format(time()))

In [ ]:
hist = model.fit(X_train, 
                       y_train, 
                       epochs = 5, 
                       batch_size = 256 , 
                       validation_data = (X_validation, y_validation), 
                       callbacks=[early_stop,
                                  checkpoint,
                                  tensorboard])
 

In [ ]:
loss, acc = model.evaluate(X_test, y_test)

In [ ]:
print(f"Accuracy: {acc*100}\nLoss: {loss}")

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("CNN.h1")

In [ ]:
from sklearn.metrics import (accuracy_score,
                             confusion_matrix, classification_report)

In [ ]:

pred = np.argmax(model.predict(X_test), axis=1)

# Classification Report
print(classification_report(pred+1, (y_test+1),
      target_names = ['1.Bog', 
                      '2.Fen',
                      '3.Marsh',
                      '4.Swamp',
                      '5.Shallow_water',
                      '6.Urban',
                      '7.Deep_water',
                      '8.Upland',
                      ]))

In [ ]:
############################################# classification confusion matrix

y_pred = np.argmax(model.predict(X_test), axis=1)

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

mat = confusion_matrix(y_test+1, y_pred+1)
#sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
mat

In [ ]:
pip install pyrsgis

In [ ]:
pip install gdal

In [ ]:
pip install matplotlib

In [ ]:
import numpy as np
import os, glob
import random
from pyrsgis import raster

In [ ]:
########################################################## Load a new multispectral image

ds, featuresAvalon = raster.read('AvalonCNN.tif')

# Generate image chips in the back-end
def CNNdataGenerator(mxBands, kSize):
    nBands, rows, cols = mxBands.shape
    margin = math.floor(kSize/2)
    mxBands = np.pad(mxBands, margin, mode='constant')[3:-3, :, :]

    features = np.empty((rows*cols, kSize, kSize,nBands))

    n = 0
    for row in range(margin, rows+margin):
        for col in range(margin, cols+margin):
            feat = mxBands[:, row-margin:row+margin+1, col-margin:col+margin+1]

            b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15, b16, b17, b18, b19, b20, b21, b22, b23, b24, b25, b26, b27, b28 = feat
            feat = np.dstack((b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15, b16, b17, b18, b19, b20, b21, b22, b23, b24, b25, b26, b27, b28))

            features[n, :, :, :] = feat
            n += 1
            
    return(features)

In [ ]:
pip install python-math

In [ ]:
import math

In [ ]:
featuresAvalon.shape

In [ ]:
###################################################################### Call the function to generate features tensor

new_features = CNNdataGenerator(featuresAvalon, kSize=7)
print(new_features.shape)
#new_features = tf.transpose(new_features, [0, 2, 3, 1])
print('Shape of the new features', new_features.shape)

In [ ]:
################################################################## Predict new data and export the probability raster

newPredicted = np.argmax(model.predict(new_features), axis=1)
#print("Shape of the predicted labels: ", newPredicted.shape)
#newPredicted = newPredicted[:,1]
newPredicted=newPredicted+1

prediction = np.reshape(newPredicted, (ds.RasterYSize, ds.RasterXSize))

In [ ]:
fig, ax=plt.subplots(figsize=(10,10))
ax.imshow(prediction)

In [ ]:
############################################################### Exporting Raster Prediction into Google Drive

outFile = ''/content/drive/MyDrive/DForest/CNN.tif''
raster.export(prediction, ds, filename=outFile, dtype='float')